In [7]:
import logging
from logging.handlers import RotatingFileHandler
import os
import asyncio
from magictables import MagicTable
from dotenv import load_dotenv

from magictables.sources import APISource

# Load environment variables
load_dotenv()

# Set up logging
log_directory = "logs"
if not os.path.exists(log_directory):
    os.makedirs(log_directory)

log_file = os.path.join(log_directory, "magictables.log")

# Create a RotatingFileHandler
file_handler = RotatingFileHandler(log_file, maxBytes=10 * 1024 * 1024, backupCount=5)
file_handler.setLevel(logging.DEBUG)

# Create a StreamHandler for console output
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it for both handlers
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Get the root logger and add both handlers
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Get the TMDb API key
TMDB_API_KEY = os.getenv("TMDB_API_KEY")

# Base URL for TMDb API
BASE_URL = "https://api.themoviedb.org/3"

async def main():
    # 1. Start with the first page of popular movies
    initial_url = f"{BASE_URL}/movie/popular?api_key={TMDB_API_KEY}&page=1"
    api_source = APISource(initial_url)
    popular_movies = await MagicTable.from_source(api_source)

    print("Popular movies (first page):")
    print(popular_movies.head())
    print(f"Total movies fetched: {len(popular_movies)}")

    # 2. Chain requests to get detailed information for each movie
    movie_details = await popular_movies.chain(
        f"{BASE_URL}/movie/{{id}}?api_key={TMDB_API_KEY}",
    )

    print("\nDetailed movie information:")
    print(movie_details.head())

    # 4. Transform the data to create a summary
    summary = await movie_details.transform(
        """
        Get the top 5 highest rated movies unique by id show columns of name and release date and relevant details for booking
        """
    )

    print("\nTop 10 movies summary:")
    print(summary)


await main()

DEBUG:root:     page  total_pages  total_results  adult  \
0       1        45285         905682  False   
1       1        45285         905682  False   
2       1        45285         905682  False   
3       1        45285         905682  False   
4       1        45285         905682  False   
..    ...          ...            ...    ...   
204     1        45285         905682  False   
205     1        45285         905682  False   
206     1        45285         905682  False   
207     1        45285         905682  False   
208     1        45285         905682  False   

                        backdrop_path         genre_ids      id  \
0    /tncbMvfV0V07UZozXdBEq4Wu9HH.jpg  [28, 80, 53, 35]  573435   
1    /tncbMvfV0V07UZozXdBEq4Wu9HH.jpg  [28, 80, 53, 35]  573435   
2    /tncbMvfV0V07UZozXdBEq4Wu9HH.jpg  [28, 80, 53, 35]  573435   
3    /tncbMvfV0V07UZozXdBEq4Wu9HH.jpg  [28, 80, 53, 35]  573435   
4    /tncbMvfV0V07UZozXdBEq4Wu9HH.jpg  [28, 80, 53, 35]  573435   
..        

Popular movies (first page):
shape: (5, 17)
┌──────┬─────────────┬──────────────┬───────┬───┬──────────────┬───────┬──────────────┬────────────┐
│ page ┆ total_pages ┆ total_result ┆ adult ┆ … ┆ title        ┆ video ┆ vote_average ┆ vote_count │
│ ---  ┆ ---         ┆ s            ┆ ---   ┆   ┆ ---          ┆ ---   ┆ ---          ┆ ---        │
│ i64  ┆ i64         ┆ ---          ┆ bool  ┆   ┆ str          ┆ bool  ┆ f64          ┆ i64        │
│      ┆             ┆ i64          ┆       ┆   ┆              ┆       ┆              ┆            │
╞══════╪═════════════╪══════════════╪═══════╪═══╪══════════════╪═══════╪══════════════╪════════════╡
│ 1    ┆ 45285       ┆ 905682       ┆ false ┆ … ┆ Bad Boys:    ┆ false ┆ 7.523        ┆ 914        │
│      ┆             ┆              ┆       ┆   ┆ Ride or Die  ┆       ┆              ┆            │
│ 1    ┆ 45285       ┆ 905682       ┆ false ┆ … ┆ Inside Out 2 ┆ false ┆ 7.6          ┆ 2083       │
│ 1    ┆ 45285       ┆ 905682       ┆ false ┆ …